# 실행 가능성 확인하기

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터

In [15]:
EPOCHS = 5

DATASET_PATH = 'dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## 단순한 모델 설정

In [12]:
def Model():
    return Sequential([Conv2D(32, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(64, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3,3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')
                      ])

## 데이터셋 불러오기

In [4]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.io.decode_png(img)
    return tf.image.convert_image_dtype(img, tf.float32)

In [5]:
# OK dataset
ok_list = glob.glob(DATASET_OK_PATTERN)
print(ok_list[:5])

ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list)) ##  [0 for _ in len(ok_list)] 와 동일

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

# FAIL dataset
fail_list = glob.glob(DATASET_FAIL_PATTERN)
print(fail_list[:5])

ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list)) ##  [0 for _ in len(fail_list)] 와 동일

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

['dataset/2/OK\\0000.png', 'dataset/2/OK\\0001.png', 'dataset/2/OK\\0002.png', 'dataset/2/OK\\0003.png', 'dataset/2/OK\\0004.png']
['dataset/2/FAIL\\0000.png', 'dataset/2/FAIL\\0001.png', 'dataset/2/FAIL\\0002.png', 'dataset/2/FAIL\\0003.png', 'dataset/2/FAIL\\0004.png']


In [6]:
ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [9]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size) # batch크기 = ds_size로 지정
ds_train = ds.take(train_size).shuffle(1024, # 버퍼크기 1024개
                                    reshuffle_each_iteration = True).batch(32)
ds_valid = ds.skip(train_size).batch(32) 

## 모델 생성 및 학습

In [16]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Train for 21 steps, validate for 9 steps
Epoch 1/5
21/21 [==============================] - 93s 4s/step - loss: 0.4750 - accuracy: 0.8805 - val_loss: 0.4428 - val_accuracy: 0.8908
Epoch 2/5
21/21 [==============================] - 90s 4s/step - loss: 0.3801 - accuracy: 0.8835 - val_loss: 0.3610 - val_accuracy: 0.8662
Epoch 3/5
21/21 [==============================] - 91s 4s/step - loss: 0.3532 - accuracy: 0.8744 - val_loss: 0.3147 - val_accuracy: 0.9261
Epoch 4/5
21/21 [==============================] - 90s 4s/step - loss: 0.3533 - accuracy: 0.8638 - val_loss: 0.3623 - val_accuracy: 0.8556
Epoch 5/5


## 결과를 이미지로 저장

In [ ]:
def mkdir(path):
    if os.path.exists(path) is FALSE:
        os.makedir(path)
mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        elif gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/TN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TP'    
        cv2.imwrite(path + '/%.f_%04d.png' %(y, index), imgs[idx].numpy() * 255)
        index += 1